<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Add WhatsApp message to contact
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Add+WhatsApp+message+to+contact:+Error+short+description">Bug report</a>

**Tags:** #hubspot #communications #whatsapp #snippet #contact #association

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-17 (Created: 2023-08-17)

**Description:** This notebook demonstrates how to add WhatsApp message to a given contact in HubSpot. It uses the communication endpoint in HubSpot.

**References:**
- [HubSpot API - Communications](https://developers.hubspot.com/docs/api/crm/communications)
- [HubSpot API - Associations v4](https://developers.hubspot.com/docs/api/crm/associations)

## Input

### Import libraries

In [1]:
import requests
import naas
from datetime import datetime, date, timezone
from pprint import pprint

### Setup variables
[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `communication_body`: This variable stores the message that will be added to the communication.
- `contact_id`: This variable stores the HubSpot contact ID that will be associated to the communication.
- `communication_datetime`: This variable stores date and time of the communication. If None communication date will be now.

In [2]:
api_key = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
communication_body = "My First WhatsApp message"
contact_id = "308551"
communication_datetime = None or datetime(2023, 8, 17, 16, 5, 5)

## Model

### Create communication

In [3]:
def create_communication(
    api_key,
    communication_body,
    contact_id,
    communication_datetime=None,
    channel_type=None,
):
    # Init
    data = []
    
    # Get the current timestamp in UTC
    if not communication_datetime:
        communication_datetime = datetime.utcnow()
        
    hs_timestamp = communication_datetime.replace(tzinfo=timezone.utc).strftime("%s") + "000"
    
    # Requests
    url = "https://api.hubapi.com/crm/v3/objects/Communications"
    payload = {
        "properties":
        {
            "hs_communication_channel_type": channel_type,
            "hs_communication_logged_from": "CRM",
            "hs_communication_body": communication_body,
            "hs_timestamp": hs_timestamp
        },
        "associations": [
            {
                "to": {"id": contact_id},
                "types": [
                    {
                        "associationCategory": "HUBSPOT_DEFINED",
                        "associationTypeId": 81 #contact
                    }
                ]
            }
        ]
    }

    headers = {
        'accept': "application/json",
        'content-type': "application/json",
        'authorization': f"Bearer {api_key}"
    }

    # Response
    res = requests.post(url, headers=headers, json=payload)
    if res.status_code == 201:
        data = res.json()
    else:
        print(res.text)
    return data

data = create_communication(api_key, communication_body, contact_id, communication_datetime, "WHATS_APP")
data

## Output

### Display result

In [4]:
if len(data) > 0:
    pprint(data.get("properties"))